# 提示词的构建指南

## 安装基础环境

首先需要安装 OpenAI 库：
```bash
pip install openai
```

In [2]:
import os

print("hello llm")
os.environ['OPENAI_API_KEY'] = 'sk-'
llm = "gpt-3.5-turbo"

hello llm


In [2]:
print("hello "+llm)
# print(os.environ.get("OPENAI_API_KEY"))

hello gpt-3.5-turbo


整个课程将以 gpt-3.5-turbo 模型为例。在此处，我们先将它封装成一个函数，您无需知道其内部机理，仅需知道调用该函数，以 Prompt 为输入参数，其将会输出对应的 Completion （回答结果）即可。

In [4]:
import openai
from openai import OpenAI

openai.api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI()

# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt: str, model: str="gpt-3.5-turbo") -> str:
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message.content

## 两个基本原则

### 原则一：编写清晰、具体的指令

您应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的操作。这将引导模型给出正确的输出，并降低您得到无关或不正确响应的可能性。清晰的指令不意味着必须简短，在许多情况下，更长的 Prompt 实际上更清晰，且提供了更多上下文，也就可能产生更详细更相关的输出。

**2.1.1 使用分隔符清晰地表示输入的不同部分**

分隔符可以是：```，""，<>，:，\<tag> \</tag>等。

您可以使用任何明显的标点符号将特定的文本部分与 Prompt 的其余部分分开。标记的形式不限，只需要让模型明确知道这是一个单独部分。使用分隔符可以有效避免提示词注入( Prompt injection )。提示词注入是指如果允许用户将某些输入添加到（开发者预定义的） Prompt 中，则所提供的指令可能会与开发者想要执行的操作相冲突，从而使 LLM 遵循用户输入的指令，而非执行开发者预期的操作。即，输入里面可能包含其他指令，会覆盖掉您的指令。对此，使用分隔符是一个不错的策略。

在以下的例子中，我们给出一段话并要求 GPT 进行总结，在该示例中我们使用 ``` 来作为分隔符。

In [18]:
text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示词与写简短的提示词混淆。\
在许多情况下，更长的提示词可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)

提供清晰、具体的指示可以引导模型朝向所需的输出，避免无关或不正确的响应。


**2.1.2 寻求结构化的输出**

输出可以是 Json、HTML 等格式。

第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析，例如，您可以在 Python 中将其读入字典或列表中。

在以下示例中，我们要求 GPT 生成三本书的标题、作者和类别，并要求 GPT 以 Json 的格式返回给我们，为便于解析，我们指定了 Json 的键。

In [19]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)

```json
[
    {
        "book_id": 1,
        "title": "The Night Circus",
        "author": "Erin Morgenstern",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "1984",
        "author": "George Orwell",
        "genre": "Dystopian"
    },
    {
        "book_id": 3,
        "title": "The Hunger Games",
        "author": "Suzanne Collins",
        "genre": "Young Adult"
    }
]
```


**2.1.3 要求模型检查是否满足条件**

如果任务包含不一定能满足的假设（条件），我们可以告诉模型先检查这些假设，如果不满足，则会指出并停止执行后续的完整流程。您还可以考虑可能出现的边缘情况及模型的应对，以避免意外的结果或错误发生。

在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答“未提供步骤”。

In [20]:
# 满足条件的输入（text中提供了步骤）
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水烧开。
第二步 - 拿一个杯子并把茶包放进去。
第三步 - 把烧开的水倒在茶包上。
第四步 - 等待几分钟，让茶叶浸泡。
第五步 - 取出茶包。
第六步 - 如果愿意，可以加入糖或牛奶调味。
第七步 - 尽情享受您的茶。


In [5]:
# 不满足条件的输入（text中未提供预期指令）
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤。


**2.1.4 提供少量示例**（少样本提示词，Few-shot prompting）

即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和祖父之间的对话的例子。孩子说，“请教我何为耐心”，祖父用下述风格的隐喻来回答。由于我们已经告诉模型要以一致的语气回答，因此现在我们问“请教我何为韧性”，由于模型已经有了这个少样本示例( few-shot example )，它将以类似的语气回答下一个任务。

In [6]:
prompt = f"""
您的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

<祖父母>: 就像大树在风雨中屹立不倒，韧性是一种坚韧不拔的品质。要学会在困难面前坚持不懈，永不放弃。只有经历过挫折和困难，才能变得更加坚强。要相信自己的能力，坚定地走向目标，不被外界的干扰所动摇。这样，你就能拥有真正的韧性。


### 2.2 给模型时间去思考

如果您发现模型推理过程过于匆忙，导致得出了错误的结论，那么您应该尝试重新构思 Prompt ，要求模型在提供最终答案之前开展**思维链**，或进行一系列相关推理（a chain or series of relevant reasoning）。换句话说，如果您给模型一个在短时间内或用少量文字无法完成的复杂任务，它的输出结果就容易出错。这种情况对人来说也是类似：如果您要求某人完成复杂的数学问题，又不给足够时间计算出答案，他们也可能会犯错误。因此，在这些情况下，您应该指示模型花更多时间思考问题，让它在任务上花费更多计算资源。

**2.2.1 指定完成任务所需的步骤**

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果。

首先我们描述了杰克和吉尔的故事，并给出提示词执行以下操作：首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和人名个数。要求输出以换行符分隔。

In [8]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：french_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
1- 兄妹在山顶井里打水时发生意外，但他们的冒险精神依然不减。
2- Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne.
3- ["Jack", "Jill"]
4- {
   "french_summary": "Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne.",
   "num_names": 2
}


上述输出仍然存在一定问题，例如，键“姓名”会被替换为法语

因此，我们将Prompt加以改进，该 Prompt 前半部分不变，同时**确切指定了输出的格式**。

In [17]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：french_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<法语摘要中的名称列表>
输出 JSON：<带有 french_summary 和 num_names 的 JSON>

文本: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
摘要: 在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水，不幸中途发生意外，但他们最终安全回家，继续冒险探索。

翻译: Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet d'une montagne, mais un accident survient en chemin, ils rentrent finalement chez eux en toute sécurité et continuent à explorer.

名称: Jack, Jill

输出 JSON: 
{
  "french_summary": "Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet d'une montagne, mais un accident survient en chemin, ils rentrent finalement chez eux en toute sécurité et continuent à explorer.",
  "num_names": 2
}


**2.2.2 指导模型在下结论之前找出一个自己的解法**

明确地指引模型在匆匆做决策之前，要自己思考出一份解决方案。有时这样会得到更好的结果。这与之前所述思想类似，即给模型时间思考。

接下来我们会给出一个问题和一份来自学生的解答，要求模型判断解答是否正确：

In [15]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为 100美元/平方英尺
- 我可以以 250美元/平方英尺的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
- 作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100000+100x
4. 总费用：100x+250x+100000+100x=450x+100000单位美元
"""
response = get_completion(prompt)
print(response)

学生的解决方案是正确的。首年运营的总费用为450x + 100,000美元。


但是注意，学生的解决方案实际上是错误的。（*维护费用项100x应为10x，总费用450x应为360x*）

我们可以通过指导模型先自行找出一个解法来解决这个问题。

在接下来这个 Prompt 中，我们要求模型先自行解决这个问题，再根据自己的解法与学生的解法进行对比，从而判断学生的解法是否正确。同时，我们给定了输出的格式要求。通过拆分任务、明确步骤，让模型有更多时间思考，有时可以获得更准确的结果。在这个例子中，学生的答案是错误的，但如果我们没有先让模型自己计算，那么可能会被误导以为学生是正确的。

In [16]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：
- 首先，自己解决问题，解决问题时列数学表达式。
- 然后将您的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。\
在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：
问题：
```
问题文本
```
学生的解决方案：
```
学生的解决方案文本
```
实际解决方案和步骤：
```
实际解决方案和步骤文本
```
学生的计算结果：
```
学生的计算结果文本
```
实际计算结果：
```
实际计算结果文本
```
学生的计算结果和实际计算结果是否相同：
```
是或否
```
学生的成绩：
```
正确或不正确
```

问题:
```
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为 100美元/平方英尺
- 我可以以 250美元/平方英尺的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
- 作为平方英尺数的函数，首年运营的总费用是多少。
```

学生的解决方案：
```
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100000+100x
4. 总费用：100x+250x+100000+100x=450x+100000单位美元
```

实际解决方案和步骤：
"""
response = get_completion(prompt)
print(response)

实际解决方案和步骤：
1. 土地费用：100美元/平方英尺 * x平方英尺 = 100x美元
2. 太阳能电池板费用：250美元/平方英尺 * x平方英尺 = 250x美元
3. 维护费用：固定费用10万美元 + 每平方英尺额外费用10美元 * x平方英尺 = 100000 + 10x美元
4. 总费用：100x + 250x + 100000 + 10x = 360x + 100000美元

学生的计算结果：
```
450x + 100000
```

实际计算结果：
```
360x + 100000
```

学生的计算结果和实际计算结果是否相同：
```
不相同
```

学生的成绩：
```
不正确
```
